# TP3

## Introduction

Dans le TD précédent vous avez définit en pseudo code plusieurs algo permettant de sérialiser/désérialiser une var de type $ImagePPM$ vers et depuis le disque dur.
Vous allez mettre en oeuvre dans ce TP la traduction du pseudo code précédemment écrit en Python. Vous allez donc lire et écrire des fichiers. Dans l'environnement que nous utilisons (un notebook JupyterLite), les fichiers qui n'ont pas une extension *.txt* sont considérés comme étant des fichiers encodés en binaire. Cela explique pourquoi vous en pouvez pas visualiser facilement le contenu des fichiers *.ppm*.

Afin de pouvoir simplement vérifier que ce que vous faites est correct, nous avons changé toutes les extensions de fichiers des images *.ppm* en *.txt*
Cela n'empèche pas de charger et afficher les images. Voir par exemple la cellule ci dessous:

## PPM

### Désérialisation

In [ ]:
from io import TextIOWrapper
import cv2
from iiwHelper import *

file : TextIOWrapper = open("./pict/Simpsons.ppm", "r")
content : list[str] = file.readlines()

width: int = int(content[1] )
height: int = int(content[2])
pixels: list[Pixel] = []

index : int = int(4)
while ((index+2) < len(content)):
    pixels.append(Pixel(int(content[index]), int(content[index+1]), int(content[index+2])))
    index = index + 3

img: ImagePPM = ImagePPM(width, height, pixels)
showImage(img)

file.close()

### Sérialisation

In [ ]:
from io import TextIOWrapper
import cv2
from iiwHelper import *

file : TextIOWrapper = open("./pict/modified/Simpsons.ppm", "w")

file.write("P3\n")
file.write(str(img.width)+"\n")
file.write(str(img.height)+"\n")
file.write(str("255\n"))

index : int = int(0)
while (index < len(img.pixels)):
    p: Pixel = img.pixels[index]
    file.write(str(p.r)+"\n")
    file.write(str(p.g)+"\n")
    file.write(str(p.b)+"\n")
    index = index + 1

print("image saved")
showImageFromPath("./pict/modified/Simpsons.ppm")

file.close()

## format IIW

### Sérialisation

In [ ]:
#v1
from io import TextIOWrapper
import cv2
from iiwHelper import *

img: ImagePPM = loadImage("./pict/miniDamier.ppm")

file : TextIOWrapper = open("./pict/modified/miniDamier.iiw", "w")

file.write("IIW\n")
file.write(str(img.width)+"\n")
file.write(str(img.height)+"\n")
file.write(str("255\n"))

previousPixel: Pixel = img.pixels[0]
nbConsecutiveSimilarPixels: int = int(1)

index : int = int(1)
while (index < len(img.pixels)):
    p: Pixel = img.pixels[index]
    if (p == previousPixel):
        nbConsecutiveSimilarPixels += 1
    else:
        file.write(str(nbConsecutiveSimilarPixels)+"\n")
        nbConsecutiveSimilarPixels = 1
        file.write(str(previousPixel.r)+"\n")
        file.write(str(previousPixel.g)+"\n")
        file.write(str(previousPixel.b)+"\n")
    previousPixel = p
    index = index + 1

file.write(str(nbConsecutiveSimilarPixels)+"\n")
file.write(str(previousPixel.r)+"\n")
file.write(str(previousPixel.g)+"\n")
file.write(str(previousPixel.b)+"\n")

file.close()

In [ ]:
#v2
from io import TextIOWrapper
import cv2
from iiwHelper import *

img: ImagePPM = loadImage("./pict/modified/valroseCompressed.txt")

file : TextIOWrapper = open("./pict/modified/valroseCompressedIIW.txt", "w")

file.write("IIW\n")
file.write(str(img.width)+"\n")
file.write(str(img.height)+"\n")
file.write(str("255\n"))

nbConsecutiveSimilarPixels: int = int(1)

index : int = int(0)
while (index+1 < len(img.pixels)):
    currentPixel: Pixel = img.pixels[index]
    nextPixel: Pixel = img.pixels[index+1]
    if (currentPixel == nextPixel):
        nbConsecutiveSimilarPixels += 1
    else:
        file.write(str(nbConsecutiveSimilarPixels)+"\n")
        nbConsecutiveSimilarPixels = 1
        file.write(str(currentPixel.r)+"\n")
        file.write(str(currentPixel.g)+"\n")
        file.write(str(currentPixel.b)+"\n")
    index = index + 1

file.write(str(nbConsecutiveSimilarPixels)+"\n")
file.write(str(img.pixels[len(img.pixels)-1].r)+"\n")
file.write(str(img.pixels[len(img.pixels)-1].g)+"\n")
file.write(str(img.pixels[len(img.pixels)-1].b)+"\n")

file.close()

### Désérialiser

In [ ]:
from io import TextIOWrapper
import cv2
from iiwHelper import *

file : TextIOWrapper = open("./pict/modified/Simpsons.iiw", "r")
content : list[str] = file.readlines()

width: int = int(content[1] )
height: int = int(content[2])
pixels: list[Pixel] = []

index : int = int(4)
while ((index+3) < len(content)):
    nbPixels: int = int(content[index])
    r: int = int(content[index+1])
    g: int = int(content[index+2])
    b: int = int(content[index+3])
    while (nbPixels > 0):
        pixels.append(Pixel(r,g,b))
        nbPixels -= 1
    index = index + 4

img: ImagePPM = ImagePPM(width, height, pixels)
showImage(img)

file.close()

### Compression avec perte

In [ ]:
from io import TextIOWrapper
import cv2
from iiwHelper import *

img: ImagePPM = loadImage("./pict/valrose.ppm")
showImage(img)


index: int = int(0)
pixels: list[Pixel] = []
nbPixelToAverage: int = int(8)

while (index < len(img.pixels)):
    r: int = int(0)
    g: int = int(0)
    b: int = int(0)
    i: int = int(0)
    borne: int = int(nbPixelToAverage)
    if((index)%img.width > img.width-nbPixelToAverage): # seulement dans 2eme partie
        borne = img.width -((index)%img.width)
    while (i < borne):   
        r += img.pixels[index+i].r
        g += img.pixels[index+i].g
        b += img.pixels[index+i].b
        i = i + 1
    p: Pixel = Pixel(r//borne,g//borne,b//borne)
    i = 0
    while (i < borne):   #pixels.extends([p]*borne)
        pixels.append(p)
        i = i + 1
    index = index+borne

res: ImagePPM = ImagePPM(img.width, img.height, pixels)
print('done')
showImage(res)
saveImage(res,'./pict/modified/valroseCompressed.txt')
#loadImage('./pict/modified/valroseCompressed.ppm')

## Format IIW++

### sérialisation

In [ ]:
from io import TextIOWrapper
import cv2
from iiwHelper import *

img: ImagePPM = loadImage("./pict/damier_propre_ascii.ppm")

file : TextIOWrapper = open("./pict/modified/damier_propre_ascii.iiw", "w")

file.write("IIW++\n")
file.write(str(img.width)+"\n")
file.write(str(img.height)+"\n")
file.write(str("255\n"))

pixelsUnique: list[Pixel] = []

index : int = int(0)
while (index < len(img.pixels)):
    p: Pixel = img.pixels[index]
    if (not (p in pixelsUnique)): # 'in' est en O(n) et peut être implémenté en parcourant la liste
        pixelsUnique.append(p)    # pixesUnique = PixelsUnique + [p]
    index = index + 1

lenPixelsUnique: int = len(pixelsUnique)
print('l\'image utilise', lenPixelsUnique,'couleurs différentes')
print('ratio:',len(img.pixels)/lenPixelsUnique)

index = 0
while (index < lenPixelsUnique):
    p: Pixel = pixelsUnique[index]
    file.write(str(p.r)+"\n")
    file.write(str(p.g)+"\n")
    file.write(str(p.b)+"\n")
    index = index + 1
file.write("-\n")

index = 0
while (index < len(img.pixels)):
    p: Pixel = img.pixels[index]
    file.write(str(pixelsUnique.index(p))+"\n") # index() est en O(n). Peut aussi être implémenté en parcourant la liste
    index = index + 1

file.close()

In [ ]:
#VOptim for test only

from io import TextIOWrapper
import cv2
from iiwHelper import *

img: ImagePPM = loadImage("./pict/valrose.ppm")

file : TextIOWrapper = open("./pict/modified/valrose.txt", "w")

file.write("IIW++\n")
file.write(str(img.width)+"\n")
file.write(str(img.height)+"\n")
file.write(str("255\n"))

pixelsUnique: dict[Pixel] = {}
keyIndex: int = int(0)
previousLenPixelUnique: int = len(pixelsUnique)

index : int = int(0)
while (index < len(img.pixels)):
    p: Pixel = img.pixels[index]
    previousLenPixelUnique = len(pixelsUnique)
    pixelsUnique[p] = keyIndex
    lenPixelsUnique: int = len(pixelsUnique)
    if(previousLenPixelUnique != lenPixelsUnique):
        keyIndex = keyIndex + 1
    index = index + 1

lenPixelsUnique: int = len(pixelsUnique)
print('l\'image utilise', lenPixelsUnique,'couleurs différentes')
print('ratio:',len(img.pixels)/lenPixelsUnique)

for p in pixelsUnique:
    file.write(str(p.r)+"\n")
    file.write(str(p.g)+"\n")
    file.write(str(p.b)+"\n")
file.write("-\n")

index = 0
while (index < len(img.pixels)):
    p: Pixel = img.pixels[index]
    file.write(str(pixelsUnique[p])+"\n") # index() est en O(n). Peut aussi être implémenté en parcourant la liste
    index = index + 1

file.close()
print('done')

## reduction image

In [ ]:
from io import TextIOWrapper
import cv2
from iiwHelper import *

img: ImagePPM = loadImage("./pict/Simpson.ppm")
ratio: float = float(0.8)

newWidth: int = int(ratio * img.width)
newHeight: int = int(ratio * img.height)



